In [1]:
from rs_graph.modeling import predict_dev_author_matches

eva_github = {
    "username": "evamaxfield",
    "name": "Eva Maxfield Brown",
    "email": "None",
}
eva_author = {
    "name": "Eva Maxfield Brown",
}
nic_github = {
    "username": "nniiicc",
    "name": "Nic Weber",
    "email": "None",
}

predictions = predict_dev_author_matches(
    [eva_github, nic_github],
    [eva_author, eva_author],
)
predictions

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['match', 'no-match']

In [2]:
import pandas as pd
from tqdm import tqdm

from rs_graph.data import (
    load_author_contributions_dataset,
    load_repo_contributors_dataset,
)

# Load the two datasets
devs = load_repo_contributors_dataset()
authors = load_author_contributions_dataset().sample(100)

print(devs.shape)
print(authors.shape)

(15731, 8)
(100, 4)


In [3]:
processed_authors = []
repo_to_author_id_lut = {}
for _, author in authors.iterrows():
    repos = [c["repo"] for c in author.contributions]

    for repo in repos:
        if repo not in repo_to_author_id_lut:
            repo_to_author_id_lut[repo] = []

        repo_to_author_id_lut[repo].append(author.author_id)

    processed_authors.append({
        "name": author["name"],
        "author_id": author.author_id,
    })

processed_authors = pd.DataFrame(processed_authors)
processed_authors.sample(3)

,name,author_id
21,R. Gelderloos,9619392
80,Jennifer A. Schmidt,2149991567
38,Andreas Selamtzis,5050126


In [4]:
# Create dataframe of dev username, name, email, repos, and co_contributors
processed_devs = []
repo_to_dev_lut = {}
for dev, group in devs.groupby("username"):
    repos = group["repo"].tolist()

    for repo in repos:
        if repo not in repo_to_dev_lut:
            repo_to_dev_lut[repo] = []

        repo_to_dev_lut[repo].append(dev)

    processed_devs.append({
        "username": dev,
        "name": group.iloc[0]["name"],
        "email": group.iloc[0]["email"],
    })

processed_devs = pd.DataFrame(processed_devs)
processed_devs.sample(3)

,username,name,email
5438,gabuzi,None,None
9804,rspavel,Robert Pavel,None
4463,db434,Daniel Bates,None


In [5]:
# Create comparison dataframe of devs and authors in the same repo
repo_and_dev_author_comparisons = []
for repo, dev_usernames in tqdm(repo_to_dev_lut.items()):
    if repo not in repo_to_author_id_lut:
        continue

    author_ids = repo_to_author_id_lut[repo]

    for dev_username in dev_usernames:
        for author_id in author_ids:
            dev_details = processed_devs.loc[
                processed_devs["username"] == dev_username
            ].iloc[0].to_dict()
            author_details = processed_authors.loc[
                processed_authors["author_id"] == author_id
            ].iloc[0].to_dict()

            # Append
            repo_and_dev_author_comparisons.append({
                "repo": repo,
                "dev_details": dev_details,
                "author_details": author_details,
            })

repo_and_dev_author_comparisons = pd.DataFrame(repo_and_dev_author_comparisons)
repo_and_dev_author_comparisons.sample(3)

100%|██████████| 2610/2610 [00:00<00:00, 2619.72it/s]


,repo,dev_details,author_details
226,https://github.com/Ouranosinc/xclim,"{'username': 'Beauprel', 'name': 'Alexis BL', ...","{'name': 'Philippe Roy', 'author_id': '2146989..."
734,https://github.com/mlpack/mlpack,"{'username': 'gboyer', 'name': 'Garry Boyer', ...","{'name': 'Ryan R. Curtin', 'author_id': '34658..."
435,https://github.com/bids-standard/pybids,"{'username': 'hjmjohnson', 'name': 'Hans Johns...","{'name': 'Michael Hanke', 'author_id': '1761354'}"


In [6]:
repo_and_dev_author_comparisons["predictions"] = predict_dev_author_matches(
    repo_and_dev_author_comparisons["dev_details"].tolist(),
    repo_and_dev_author_comparisons["author_details"].tolist(),
)
repo_and_dev_author_comparisons["predictions"].value_counts()

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

no-match    994
match        51
Name: predictions, dtype: int64

In [7]:
repo_and_dev_author_comparisons.loc[
    repo_and_dev_author_comparisons.predictions == "match"
][["dev_details", "author_details"]]

,dev_details,author_details
0,"{'username': '1313e', 'name': 'Ellert van der ...","{'name': 'Ellert van der Velden', 'author_id':..."
1,"{'username': '1313e', 'name': 'Ellert van der ...","{'name': 'Ellert van der Velden', 'author_id':..."
48,"{'username': 'asalmgren', 'name': 'Ann Almgren...","{'name': 'A. Almgren', 'author_id': '2896475'}"
86,"{'username': 'asalmgren', 'name': 'Ann Almgren...","{'name': 'A. Almgren', 'author_id': '2896475'}"
108,"{'username': 'bbenneke', 'name': 'Björn Bennek...","{'name': 'B. Benneke', 'author_id': '7176439'}"
128,"{'username': 'williamjameshandley', 'name': 'W...","{'name': 'Will Handley', 'author_id': '1453931..."
139,"{'username': 'williamjameshandley', 'name': 'W...","{'name': 'Will Handley', 'author_id': '1453931..."
173,"{'username': 'agosiewska', 'name': 'Alicja Gos...","{'name': 'Alicja Gosiewska', 'author_id': '514..."
225,"{'username': 'Balinus', 'name': 'Philippe Roy'...","{'name': 'Philippe Roy', 'author_id': '2146989..."
269,"{'username': 'bgupt', 'name': 'Brajesh Gupt', ...","{'name': 'Brajesh Gupt', 'author_id': '15964560'}"
